# <a id='toc1_'></a>[Statistical tests and tools playground](#toc0_)

<!-- vscode-jupyter-toc --><a id='toc0_'></a>    
- [Statistical tests and tools playground](#toc1_)    
  - [Import data](#toc1_1_)    
  - [Bootstrapping](#toc1_2_)    
      - [_Estimate standard error and confidence interval for a given statistic_](#toc1_2_1_1_)    
    - [Bootstrap standard error for a statistic](#toc1_2_2_)    
    - [Boostrap confidence intervals for a statistic](#toc1_2_3_)    
      - [Mean (or other single-sample statistics)](#toc1_2_3_1_)    
      - [Corr coefficient](#toc1_2_3_2_)    
  - [T-test & Permutation tests](#toc1_3_)    
      - [_Comparison between means of 2 groups_](#toc1_3_1_1_)    
  - [Bonus: Permutation tests for other multi-sample statistics (eg. corr coefficient)](#toc1_4_)    
      - [_Check p-value of multi-sample statistic vs null hypothesis_](#toc1_4_1_1_)    
  - [ANOVA & OLS models](#toc1_5_)    
      - [_Comparison between means of 3+ groups_](#toc1_5_1_1_)    
  - [Proportions tests (Z-test)](#toc1_6_)    
      - [_Compare proportions of 2 different groups_](#toc1_6_1_1_)    
  - [Chi-square (contigency) tests](#toc1_7_)    
      - [_Comparison between the frequencies of multiple groups ('count' data)_](#toc1_7_1_1_)    
  - [Exact tests for proportions (Fisher, Barnard, Boschloo)](#toc1_8_)    
      - [_Comparison bw frequencies of 2 groups (2x2 contigency tables) with very low sample size_](#toc1_8_1_1_)    
  - [Sample size calculations](#toc1_9_)    
      - [_Given effect size to be detected and power, solve for sample size of T or Z test_](#toc1_9_1_1_)    
  - [Extra infos & links](#toc1_10_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [2]:
import stats_utils as stats
from scipy.stats import bootstrap, permutation_test, ttest_ind, f_oneway, kruskal, alexandergovern, tukey_hsd, pearsonr, chi2_contingency, fisher_exact, barnard_exact, boschloo_exact
import statsmodels.api as sm
from statsmodels.stats.proportion import proportions_ztest, proportion_effectsize
from statsmodels.stats.power import tt_ind_solve_power, zt_ind_solve_power
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import pandas as pd
import numpy as np

## <a id='toc1_1_'></a>[Import data](#toc0_)

In [2]:
# get some data
iris = pd.read_csv(
    'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [3]:
# get some data
titanic = pd.read_csv(
    'https://storage.googleapis.com/tf-datasets/titanic/train.csv')
titanic #.head()

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
...,...,...,...,...,...,...,...,...,...,...
622,0,male,28.0,0,0,10.5000,Second,unknown,Southampton,y
623,0,male,25.0,0,0,7.0500,Third,unknown,Southampton,y
624,1,female,19.0,0,0,30.0000,First,B,Southampton,y
625,0,female,28.0,1,2,23.4500,Third,unknown,Southampton,n


## <a id='toc1_2_'></a>[Bootstrapping](#toc0_)
#### <a id='toc1_2_1_1_'></a>[_Estimate standard error and confidence interval for a given statistic_](#toc0_)

### <a id='toc1_2_2_'></a>[Bootstrap standard error for a statistic](#toc0_)

In modern statistics, the bootstrap has become the standard way to estimate standard error and other accuracy measures for estimators (eg. bias, confidence intervals, etc.). It can be used for virtually any statistic and does not rely on the central limit theorem or other distributional assumptions.

In [72]:
### try out my bootstrapp_statistic_std_error (from utils) ###

# df = iris[iris["species"] == "setosa"]
# column_name = "sepal_length"
df = titanic
column_name = "fare"
statistic = np.mean

original, bias, std_error = stats.bootstrap_statistic_std_error(
    df, column_name, statistic, 10000)

# print results
print('Bootstrap Statistics:')
print(f'original: {original}')
print(f'bias: {bias}')
print(f'std. error: {std_error}')
if statistic is np.mean:
    # if using 'mean' you can also compute the mathematical approximation of the standard error
    std_error_approx = df[column_name].std(
    ) / (df[column_name].shape[0] ** (1/2))
    print(f'std. error with stat approx formula: {std_error_approx}')

Function bootstrap_statistic_std_error executed in 9.7193 seconds
Bootstrap Statistics:
original: 34.385398564593245
bias: 0.002295145630114348
std. error: 2.1751733214000333
std. error with stat approx formula: 2.1804233291368984


The standard error can also be computed using Scipy's bootstrap method (see below for example).

### <a id='toc1_2_3_'></a>[Boostrap confidence intervals for a statistic](#toc0_)

#### <a id='toc1_2_3_1_'></a>[Mean (or other single-sample statistics)](#toc0_)

In [5]:
### try out my boostrap_statistic_with_confidence_interval ###

# df = iris[iris["species"] == "setosa"]
# column_name = "sepal_length"
df = titanic
column_name = "fare"
statistic = np.mean

estimate, lower_bound, upper_bound = stats.boostrap_statistic_with_confidence_interval(
    df, column_name, statistic, 0.95, 10000)

# print results
print('\nBootstrap Statistics: \n------------')
print(f'estimate: {estimate}')
print(f'CI lower_bound: {lower_bound}')
print(f'CI upper_bound: {upper_bound} \n')

Function boostrap_statistic_with_confidence_interval executed in 1.5230 seconds

Bootstrap Statistics: 
------------
estimate: 34.39326916602862
CI lower_bound: 30.27921544657093
CI upper_bound: 38.81992862440188 



In [6]:
# try to get the same CIs by adding/removing 2*(estimated standard error) from the mean
    # doesn't yield exactly the same result so be careful when using this approximation (this relies on the normality assumption for the sampling distribution)
print("Lower (test): ", estimate - 2*std_error)
print("Upper (test): ", estimate + 2*std_error)

Lower (test):  30.085216493418507
Upper (test):  38.70132183863873


In [7]:
# compare (also time-wise) to scipy's boostrap method
    # which also returns the estimated standard error
    # and is about 10x faster :D
data = (df[column_name],)
res = bootstrap(data, np.mean, confidence_level=0.95, method='percentile', # method can also be 'BCa' (default) and 'basic'
                n_resamples=10000)
ci_l, ci_u = res.confidence_interval
print(f'Scipy CI lower_bound: {ci_l}')
print(f'Scipy CI upper_bound: {ci_u}')

Scipy CI lower_bound: 30.22956383971292
Scipy CI upper_bound: 38.82376676236044


The CI here (with the percentile method) is basically the same as we originally bootstrapped above.

#### <a id='toc1_2_3_2_'></a>[Corr coefficient (or other paired-sample statistics)](#toc0_)

In [8]:
def pearson_r(x, y):
    return pearsonr(x, y)[0]

In [9]:
res = bootstrap((titanic['age'], titanic['fare']), pearson_r, vectorized=False, paired=True, confidence_level=0.95, method='BCa')
print(res.confidence_interval)

ConfidenceInterval(low=0.05730125741647129, high=0.1919310919951091)


## <a id='toc1_3_'></a>[T-test & Permutation tests](#toc0_)
#### <a id='toc1_3_1_1_'></a>[_Comparison between means of 2 groups_](#toc0_)

**_T-test vs Permutation tests_**

An advantage of permutation tests is that they do not require any assumptions about the population parameters, such as the mean or variance, or the distribution of the underlying data. This makes them more robust and generally applicable to a wider range of data. On the other hand, t-tests do require assumptions about the population parameters and on the samples' distribution and variance, which can make them less reliable in certain situations.

In addition, Permutation tests exist for any test statistic, regardless of whether or not its distribution is known. Thus one is always free to choose the statistic which best discriminates between hypothesis and alternative and which minimizes losses.

**Try my implementation of a permutation test (from utils):**

In [21]:
# main example (will be also used below)
observed_diff, pvalue = stats.permutation_test_statistic_two_groups(df=titanic, output_col_name="n_siblings_spouses", 
                                                                  groups_col_name="class", treatment_group_name="First", 
                                                                  control_group_name="Second",  nrepeat=5000)
print(
    f"Observed difference (treatment - control): {observed_diff} \np-value: {pvalue}")

Function permutation_test_statistic_two_groups executed in 7.1684 seconds
Observed difference (treatment - control): 0.04813549249740012 
p-value: 0.217


In [4]:
# now try again with optimised (vectorised) version of my same function
    # compare the execution time! Almost 10x faster
observed_diff, pvalue = stats.optimised_permutation_test_statistic_two_groups(df=titanic, output_col_name="n_siblings_spouses", 
                                                                  groups_col_name="class", treatment_group_name="First", 
                                                                  control_group_name="Second",  nrepeat=5000)
print(
    f"Observed difference (treatment - control): {observed_diff} \np-value: {pvalue}")

Function optimised_permutation_test_statistic_two_groups executed in 0.1092 seconds
Observed difference (treatment - control): 0.04813549249740012 
p-value: 0.27


**Compare against scipy's version**

In [5]:
# prep
x, y = titanic.loc[titanic['class'] == 'Second', ['n_siblings_spouses']], titanic.loc[titanic['class'] == 'First', ['n_siblings_spouses']]

def statistic(x, y, axis):
    # y - x
    return np.mean(y, axis=axis) - np.mean(x, axis=axis)

In [6]:
# run scipy's perm test
    # it executes in roughly 0.1s, so very similar to my optimised implementation!
res = permutation_test((x, y), statistic, vectorized=True,
                       n_resamples=5000, alternative='greater')
print(f"Observed difference (y - x): {res.statistic} \np-value: {res.pvalue}")

Observed difference (y - x): [0.04813549] 
p-value: [0.27934413]


In [ ]:
# you can also plot the resulting null distribution of the test statistic

"""
import matplotlib.pyplot as plt
plt.hist(res.null_distribution, bins=50)
plt.title("Permutation distribution of test statistic")
plt.xlabel("Value of Statistic")
plt.ylabel("Frequency")
plt.show()
"""

**Compare against t-test results**

In [36]:
# t-test assuming no equal variances (Welch’s t-test)
t_res = ttest_ind(y, x, equal_var=False, 
                    permutations=None, alternative='greater')
print(f"p-value: {t_res.pvalue}")

p-value: [0.24960374]


In [37]:
# two-sided t-test
    # -> doubles the smallest p-value bw 'greater' and 'less', so more conservative
ts_t_res = ttest_ind(y, x, equal_var=False, 
                    permutations=None, alternative='two-sided')
print(f"p-value: {ts_t_res.pvalue}")

p-value: [0.49920748]


In [38]:
# t-test with permutations
    # it generates a distribution of the t-statistic under the null hypothesis via permutations, instead of using the t-distribution
t_perm_res = ttest_ind(y, x, equal_var=False,
                  permutations=5000, alternative='greater')
print(f"p-value: {t_perm_res.pvalue}")

p-value: [0.25674865]


Note on the above: an important assumption behind a permutation test is that the observations are exchangeable under the null hypothesis. A consequence of this assumption is that tests of difference in location (like a permutation t-test) require equal variance under the normality assumption. In this respect, the permutation t-test shares the same weakness as the classical Student's t-test (the Behrens–Fisher problem). A third alternative in this situation is to use a bootstrap-based test. Good (2005) explains the difference between permutation tests and bootstrap tests the following way: "Permutations test hypotheses concerning distributions; bootstraps test hypotheses concerning parameters. As a result, the bootstrap entails less-stringent assumptions." Bootstrap tests are not exact. In some cases, a permutation test based on a properly studentized statistic can be asymptotically exact even when the exchangeability assumption is violated. [[6]](https://projecteuclid.org/journals/annals-of-statistics/volume-41/issue-2/Exact-and-asymptotically-robust-permutation-tests/10.1214/13-AOS1090.full)

In [45]:
# We can also check the power of our test given the variables in place!
    # Power is the probability that the test correctly rejects the Null Hypothesis if the Alternative Hypothesis is true.

effect_size = abs(np.mean(y) - np.mean(x))
var_x = np.var(x)
var_y = np.var(y)
pooled_std = np.sqrt((var_x + var_y) / 2)
std_effect_size = effect_size / pooled_std
nobs_x = len(x)
ratio = len(y) / len(x)

t_power = tt_ind_solve_power(effect_size=std_effect_size, nobs1=nobs_x, alpha=0.05, power=None, ratio=ratio, alternative='two-sided')
# print("Power of specified t-test: {:.1%}".format(t_power)) #TODO: check why it now throws a formatting error
print(f"Power of specified t-test: {t_power}")

Power of specified t-test: n_siblings_spouses    0.103442
dtype: float64


**Try against OLS model**

In [46]:
# compare t-test results vs simple OLS model
    # -> the p-value of the class coefficient is (basically) the same as the two-sided t-test results
    # -> only difference is that the OLS's t-test assumes equal variances (less precise!)
df_ols = titanic[titanic['class'] != 'Third']
t_model = ols('n_siblings_spouses ~ Q("class")', data=df_ols).fit()
t_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     n_siblings_spouses   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.4504
Date:                Fri, 09 Jun 2023   Prob (F-statistic):              0.503
Time:                        00:35:34   Log-Likelihood:                -259.99
No. Observations:                 286   AIC:                             524.0
Df Residuals:                     284   BIC:                             531.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.4340      0.048      9.080      0.000       0.340       0.528
Q("class")[T.Second]    -0.0481      0.072     -0.671      0.503      -0.189       0.093
==============================================================================
Omnibus:                       81.249   Durbin-Watson:                   1.874
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              167.275
Skew:                           1.445   Prob(JB):                     4.75e-37
Kurtosis:                       5.385   Cond. No.                         2.51
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## <a id='toc1_4_'></a>[Bonus: Permutation tests for other multi-sample statistics (eg. corr coefficient)](#toc0_)
#### <a id='toc1_4_1_1_'></a>[_Check p-value of multi-sample statistic vs null hypothesis_](#toc0_)

In [47]:
def pearson_r(x, y):
    return pearsonr(x, y).statistic

In [49]:
# same corr coefficient computed above in the bootstrapping section

corr_res = permutation_test((titanic['age'], titanic['fare']), pearson_r, vectorized=False,
                       permutation_type='pairings', # This permutation type is appropriate for association/correlation tests with statistics such as Spearman’s , Kendall’s , and Pearson’s .
                       alternative='two-sided')
# r, pvalue, null = corr_res.statistic, corr_res.pvalue, corr_res.null_distribution
print(f"Observed correlation: {corr_res.statistic} \np-value: {corr_res.pvalue}")

Observed correlation: 0.11928722927934457 
p-value: 0.0018


## <a id='toc1_5_'></a>[ANOVA & OLS models](#toc0_)
#### <a id='toc1_5_1_1_'></a>[_Comparison between means of 3+ groups_](#toc0_)

Try permutation-ANOVA test for difference of means:

In [50]:
# trying with a permutation-ANOVA test
x, y, z = titanic.loc[titanic['class'] == 'First', 'n_siblings_spouses'], titanic.loc[titanic['class'] == 'Second', 
    'n_siblings_spouses'], titanic.loc[titanic['class'] == 'Third', 'n_siblings_spouses']

def anova_statistic(x, y, z):
    # var of the groups' means
    return np.var([np.mean(x), np.mean(y), np.mean(z)])

# run perm test
anova_perm_res = permutation_test((x, y, z), anova_statistic, vectorized=False,
                       n_resamples=5000, alternative='greater')
print(f"Observed variance of means: {anova_perm_res.statistic} \np-value: {anova_perm_res.pvalue}")

Observed variance of means: 0.013943394715751333 
p-value: 0.0653869226154769


Try scipy's f_oneway test:

In [51]:
# one-way ANOVA F test
f_res = f_oneway(x, y, z)
print(f"p-value: {f_res.pvalue}")

p-value: 0.02804501692073304


The ANOVA test has important assumptions that must be satisfied in order for the associated p-value to be valid.

- The samples are independent.
- Each sample is from a normally distributed population.
- The population standard deviations of the groups are all equal. This property is known as homoscedasticity.

If these assumptions are not true for a given set of data, it may still be possible to use the Kruskal-Wallis H-test:

In [55]:
# Compute the Kruskal-Wallis H-test for independent samples.
# The Kruskal-Wallis H-test tests the null hypothesis that the population median of all of the groups are equal. It is a non-parametric version of ANOVA.
kruskal_res = kruskal(x, y, z)
print(f"p-value: {kruskal_res.pvalue}")

p-value: 0.5985709648385751


or the Alexander Govern test:

In [56]:
# The Alexander-Govern approximation tests the equality of k independent means in the face of heterogeneity of variance.

# The use of this test still relies on several assumptions.
    # The samples are independent.
    # Each sample is from a normally distributed population.
    # Unlike f_oneway, this test does not assume on homoscedasticity, instead relaxing the assumption of equal variances.

alexandergovern_res = alexandergovern(x, y, z)
print(f"p-value: {alexandergovern_res.pvalue}")

p-value: 0.014311269992021343


although with some loss of power.

One-way ANOVA table with OLS model:

In [40]:
# same but expanded results as f_oneway above
ow_model = ols('n_siblings_spouses ~ Q("class")', data=titanic).fit()
anova_table = sm.stats.anova_lm(ow_model, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
"Q(""class"")",9.447149,2.0,3.594492,0.028045
Residual,820.007397,624.0,NaN,NaN


In [43]:
# compare to linear reg output:
    # -> PR(>F) in anova table above and Prob(F stat) below, ie the p value of the overall comparison, are the same! (as only 1 indep. variable)
    # intercept refers to the "first" class
ow_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     n_siblings_spouses   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     3.594
Date:                Fri, 18 Nov 2022   Prob (F-statistic):             0.0280
Time:                        15:30:30   Log-Likelihood:                -973.81
No. Observations:                 627   AIC:                             1954.
Df Residuals:                     624   BIC:                             1967.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.4340      0.091      4.773      0.000       0.255       0.612
Q("class")[T.Second]    -0.0481      0.136     -0.353      0.724      -0.316       0.220
Q("class")[T.Third]      0.2229      0.110      2.025      0.043       0.007       0.439
==============================================================================
Omnibus:                      502.348   Durbin-Watson:                   2.093
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8518.282
Skew:                           3.562   Prob(JB):                         0.00
Kurtosis:                      19.593   Cond. No.                         4.48
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Two-ways ANOVA table with OLS model:

In [57]:
# type 2: more power, assumes no significant interaction effect between indep variables
tw_model = ols('n_siblings_spouses ~ Q("class") + sex', data=titanic).fit()
anova_table = sm.stats.anova_lm(tw_model, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
"Q(""class"")",11.927261,2.0,4.566602,0.010744
sex,6.417293,1.0,4.913990,0.027000
Residual,813.590104,623.0,NaN,NaN


In [58]:
# compare to linear reg output:
    # -> p-value for "sex" is the same in both outputs as it has only one level (male). For "class" it's more intricate
tw_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     n_siblings_spouses   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     4.049
Date:                Fri, 09 Jun 2023   Prob (F-statistic):            0.00724
Time:                        12:41:14   Log-Likelihood:                -971.34
No. Observations:                 627   AIC:                             1951.
Df Residuals:                     623   BIC:                             1968.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.5561      0.106      5.243      0.000       0.348       0.764
Q("class")[T.Second]    -0.0479      0.136     -0.353      0.725      -0.315       0.219
Q("class")[T.Third]      0.2577      0.111      2.325      0.020       0.040       0.475
sex[T.male]             -0.2157      0.097     -2.217      0.027      -0.407      -0.025
==============================================================================
Omnibus:                      506.664   Durbin-Watson:                   2.082
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8794.831
Skew:                           3.598   Prob(JB):                         0.00
Kurtosis:                      19.878   Cond. No.                         5.25
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [59]:
# type 3 ANOVA: less power, works with interaction effect
    # p-value of "sex" similar as above, as interaction determined not significant at all
tw_model_type3 = ols('n_siblings_spouses ~ Q("class") * sex', data=titanic).fit()
anova_table_type3 = sm.stats.anova_lm(tw_model_type3, typ=3)
anova_table_type3

,sum_sq,df,F,PR(>F)
Intercept,20.927536,1.0,15.973745,0.000072
"Q(""class"")",4.367950,2.0,1.667003,0.189656
sex,1.661918,1.0,1.268523,0.260479
"Q(""class""):sex",0.005071,2.0,0.001935,0.998066
Residual,813.585033,621.0,NaN,NaN


In [60]:
# compare to linear reg expanded output:
    # -> p_value for "sex" is the same (as only one level, as above)
    # output is broken down more, down to the single levels of each category
tw_model_type3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     n_siblings_spouses   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     2.423
Date:                Fri, 09 Jun 2023   Prob (F-statistic):             0.0344
Time:                        13:06:41   Log-Likelihood:                -971.34
No. Observations:                 627   AIC:                             1955.
Df Residuals:                     621   BIC:                             1981.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            0.5507      0.138      3.997      0.000       0.280       0.821
Q("class")[T.Second]                -0.0416      0.207     -0.201      0.841      -0.448       0.365
Q("class")[T.Third]                  0.2665      0.182      1.465      0.143      -0.091       0.624
sex[T.male]                         -0.2063      0.183     -1.126      0.260      -0.566       0.153
Q("class")[T.Second]:sex[T.male]    -0.0111      0.275     -0.041      0.968      -0.551       0.529
Q("class")[T.Third]:sex[T.male]     -0.0141      0.230     -0.062      0.951      -0.466       0.438
==============================================================================
Omnibus:                      506.675   Durbin-Watson:                   2.082
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8795.430
Skew:                           3.598   Prob(JB):                         0.00
Kurtosis:                      19.879   Cond. No.                         13.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

An ANOVA is used to determine whether or not there is a statistically significant difference between the means of three or more independent groups.

If the overall p-value from the ANOVA table is less than some significance level, then we have sufficient evidence to say that at least one of the means of the groups is different from the others.

However, this doesn’t tell us which groups are different from each other. It simply tells us that not all of the group means are equal. In order to find out exactly which groups are different from each other, we must conduct a post hoc test.

One of the most commonly used post hoc tests is Tukey’s Test, which allows us to make pairwise comparisons between the means of each group while controlling for the family-wise error rate.

In [61]:
tukey = pairwise_tukeyhsd(endog=titanic['n_siblings_spouses'],
                          groups=titanic['class'],
                          alpha=0.05)

# display results
print(tukey)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
 First Second  -0.0481 0.9337 -0.3686 0.2724  False
 First  Third   0.2229 0.1071 -0.0357 0.4816  False
Second  Third   0.2711 0.0602 -0.0089  0.551  False
---------------------------------------------------


In [ ]:
# Why can you get a significant result from the ANOVA test and not significant differences in Tukey's HSD?
    # https://stats.stackexchange.com/questions/16665/how-can-i-get-a-significant-overall-anova-but-no-significant-pairwise-difference
    # https://stats.stackexchange.com/questions/573423/anova-significant-interaction-but-not-significant-post-hoc
    # http://www.pmean.com/05/TukeyTest.html

    # In short:
        # This is mainly due to the sensitivity of ANOVA (greater than the pairwise test sensitivity). 
        # Then, ANOVA detect lower variability around mean when pairwise test hardly distinguishes between the pair's mean. 

# (Bonus) On the other hand, do you really need to run a global test (ANOVA) before post-hoc tests?
    # https://stats.stackexchange.com/questions/9751/do-we-need-a-global-test-before-post-hoc-tests
    # In short: not really, but generally better

In [51]:
# Compare to scipy's version
    # same result but full matrix
hsd_res = tukey_hsd(x, y, z)
print(hsd_res)

Tukey's HSD Pairwise Group Comparisons (95.0% Confidence Interval)
Comparison  Statistic  p-value  Lower CI  Upper CI
 (0 - 1)      0.048     0.934    -0.272     0.369
 (0 - 2)     -0.223     0.107    -0.482     0.036
 (1 - 0)     -0.048     0.934    -0.369     0.272
 (1 - 2)     -0.271     0.060    -0.551     0.009
 (2 - 0)      0.223     0.107    -0.036     0.482
 (2 - 1)      0.271     0.060    -0.009     0.551



Note: the use of Tukey's HSD test relies on several assumptions.

- The observations are independent within and among groups.
- The observations within each group are normally distributed.
- The distributions from which the samples are drawn have the same finite variance.

## <a id='toc1_6_'></a>[Proportions tests (Z-test)](#toc0_)
#### <a id='toc1_6_1_1_'></a>[_Compare proportions of 2 different groups_](#toc0_)

**_Z-test vs Permutation tests_**

A z-test for a test of proportions between two groups is used when the following conditions are met:

1. Sample size is large enough for z to be approximately normally distributed. Rule of thumb:
    - Significance test: number of successes and number of failures are each 5 or more in both sample groups
    - Regular (large sample) 90%, 95%, or 99% confidence interval: number of successes and number of failures are each 10 or more in both sample groups
    - Plus four 90%, 95%, or 99% confidence interval: sample sizes of both groups are 5 or more
2. Group 1 sample is a simple random sample (SRS) from population 1, group 2 sample is an independent SRS from population 2. That is, within and between groups, observations are independent of one another.

A permutation test, on the other hand, does not require any assumptions about the underlying distribution of the data and can be used with smaller sample sizes. It works by simulating the sampling process many times and computing the test statistic for each simulated sample. The p-value is then calculated as the fraction of simulated samples that have a test statistic that is at least as extreme as the one observed in the actual data. If the above conditions are not met, you should use instead a permutation test.

However, a permutation test is generally less powerful than a z-test or a chi-square test, so it may not be able to detect small differences between the means or proportions of the two groups. In addition, a permutation test can be computationally intensive, especially for large sample sizes, so it may not be practical in some situations.

**_Z-test vs Chi-square test_**

- The Z-test is used when comparing the difference in population proportions between 2 groups. 
- The Chi-square test is used when comparing the difference in population proportions between 2 or more groups or when comparing a group with a value.

A chi-square test for equality of two proportions is exactly the same thing as a z-test. The chi-squared distribution with one degree of freedom is just that of a normal deviate, squared. You're basically just repeating the chi-squared test on a subset of the contingency table. 

In [62]:
# Using Z-test 
    # better than T-test when working with proportions since the standard deviation is known 
    # (standard deviation of a proportion is a function of the proportion itself)

count_positives = np.array([5, 12])
nobs_tot = np.array([83, 99])

stat, pval = proportions_ztest(count_positives, nobs_tot, alternative='two-sided')
print('P-value: {0:0.3f}'.format(pval))

P-value: 0.159


In [63]:
# trial from chatGPT
    # same thing, plus it's computing the 95% CI

# import the necessary libraries
from scipy.stats import norm # rest already imported

# define the sample sizes and number of successes for each group
n1 = 100
n2 = 50
successes1 = 25
successes2 = 15

# calculate the proportions for each group
prop1 = successes1 / n1
prop2 = successes2 / n2

# compute the test statistic and p-value using the proportions_ztest function
z_stat, pval = proportions_ztest([successes1, successes2], [n1, n2], alternative='two-sided')

# interpret the results
if pval < 0.05:
    print("The difference in proportions is statistically significant.")
else:
    print("The difference in proportions is not statistically significant.")

# compute the 95% confidence interval for the difference in proportions
pdiff = prop1 - prop2
z_critical = norm.ppf(0.975)  # get the critical value of the normal distribution at 95% confidence
low, high = pdiff - z_critical * np.sqrt(prop1 * (1 - prop1) / n1 + prop2 * (1 - prop2) / n2), pdiff + z_critical * np.sqrt(prop1 * (1 - prop1) / n1 + prop2 * (1 - prop2) / n2)
print("The 95% confidence interval for the difference in proportions is ({:.3f}, {:.3f})".format(low, high))


The difference in proportions is not statistically significant.
The 95% confidence interval for the difference in proportions is (-0.203, 0.103)


## <a id='toc1_7_'></a>[Chi-square (contigency) tests](#toc0_)
#### <a id='toc1_7_1_1_'></a>[_Comparison between the frequencies of multiple groups ('count' data)_](#toc0_)

In [64]:
# A made-up example for Scipy's method
obs = np.array([[15, 10, 25], [40, 20, 30]])

index_values = ['click', 'no_click']
column_values = ['group_a', 'group_b', 'group_c']
df_obs = pd.DataFrame(data=obs,
                      index=index_values,
                      columns=column_values)
print(df_obs)

          group_a  group_b  group_c
click          15       10       25
no_click       40       20       30


In [65]:
g, p, dof, expctd = chi2_contingency(df_obs)
print("P-value: ", p)

P-value:  0.1317385467621491


## <a id='toc1_8_'></a>[Exact tests for proportions (Fisher, Barnard, Boschloo)](#toc0_)
#### <a id='toc1_8_1_1_'></a>[_Comparison bw frequencies of 2 groups (2x2 contigency tables) with very low sample size_](#toc0_)

The chi-square distribution or z-test are a good approximation of the shuffled resampling test, except when counts are extremely low (single digits, especially five or fewer). In such cases, the resampling procedure will yield more accurate p-values. 

In fact, most statistical software has a procedure to actually enumerate all the possible rearrangements (permutations) that can occur, tabulate their frequencies, and determine exactly how extreme the observed result is. This is called Fisher’s exact test after the great statistician R. A. Fisher.

In addition, more recent tests such as "Barnard exact" and "Boschloo exact" are more powerful than Fisher's.

Note, however, that many packages provide the results of these exact tests for 2 × 2 contingency tables but not for bigger contingency tables with more rows or columns. For Fisher's exact test of bigger contingency tables, we can use web pages providing such analyses. For example, the web page ‘Social Science Statistics’ (http://www.socscistatistics.com/tests/chisquare2/Default2.aspx) permits performance of Fisher exact test for up to 5 × 5 contingency tables.

In [66]:
# These exact tests only work on 2x2 contigency tables
df_obs_cut = df_obs[['group_a', 'group_b']]

_, fisher_p = fisher_exact(df_obs_cut, alternative='two-sided')
barnard_p = barnard_exact(df_obs_cut, alternative="two-sided").pvalue
boschloo_p = boschloo_exact(df_obs_cut, alternative="two-sided").pvalue

print("Fischer p-value: ", fisher_p)
print("Barnard p-value: ", barnard_p)
print("Boschloo p-value: ", boschloo_p)

Fischer p-value:  0.6219813684237752
Barnard p-value:  0.6124696915520649
Boschloo p-value:  0.5721916140286106


## <a id='toc1_9_'></a>[Sample size calculations](#toc0_)
#### <a id='toc1_9_1_1_'></a>[_Given effect size to be detected and power, solve for sample size of T or Z test_](#toc0_)

**_T-Test between means:_**

In [67]:
# Sample size is sensitive to the size and variability of the difference between groups, and tolerance to Type I and II errors.

mean_diff = 0.1
sd_diff = 1 # the st.dev of the difference or the pooled standard deviation of the 2 samples
std_effect_size = mean_diff / sd_diff # standardized effect size, difference between the two means divided by the standard deviation. It has to be positive.

means_sample_size = tt_ind_solve_power(effect_size=std_effect_size, nobs1=None, alpha=0.05, 
                                        power=0.8, ratio=1.0, alternative='two-sided')
print(
    "Sample size required to detect the specified std effect size of {:.2f}: \n{:.2f} \n(for each group)".format(std_effect_size, means_sample_size))

Sample size required to detect the specified std effect size of 0.10: 
1570.73 
(for each group)


In [68]:
# It is easy to see that changes in the standardised mean difference we want to detect will change the sample size. 
# For example, for a mean difference of 0.1 as above, sample size increases significantly as standard deviation of the difference increases:

for sd in [0.4, 0.8, 1.6]:
    n = tt_ind_solve_power(effect_size=mean_diff/sd, alpha=0.05,
                           power=0.8, ratio=1, alternative='two-sided')
    print('Number in *each* group when SD is {:<4.1f}: {:.2f}'.format(sd, n))

Number in *each* group when SD is 0.4 : 252.13
Number in *each* group when SD is 0.8 : 1005.62
Number in *each* group when SD is 1.6 : 4019.58


**_Z-Test between proportions:_**

In [69]:
# define the 2 proportions:
prop1 = 0.5
prop2 = 0.75

# compute effect size and required sample size
es = proportion_effectsize(prop1=prop1, prop2=prop2)
prop_sample_size = zt_ind_solve_power(effect_size=es, nobs1=None, alpha=0.05, power=0.9, ratio=1.0, alternative='two-sided')
print("Sample size required to detect the specified difference in proportions: \n{:.2f} \n(for each group)".format(prop_sample_size))

Sample size required to detect the specified difference in proportions: 
76.65 
(for each group)


## <a id='toc1_10_'></a>[Extra infos & links](#toc0_)

How are regression, the t-test, and the ANOVA all versions of the general linear model?
- https://stats.stackexchange.com/questions/59047/how-are-regression-the-t-test-and-the-anova-all-versions-of-the-general-linear
- https://www.researchgate.net/post/Can-anyone-help-me-to-get-the-core-differences-between-regression-model-and-ANOVA-model#view=54f420f5d685cc07258b46ea
- https://stats.stackexchange.com/questions/175246/why-is-anova-equivalent-to-linear-regression

From the mathematical point of view, linear regression and ANOVA are identical: both break down the total variance of the data into different “portions” and verify the equality of these “sub-variances” by means of a test (“F” Test). What can be added is that, in both techniques the dependent variable is a continuous one, but in the ANOVA analysis the independent variable can be exclusively categorical variable, while in the regression can be used both categorical and continuous independent variables. Thus, ANOVA can be considered as a case of a linear regression in which all predictors are categorical.
- https://www.statsimprove.com/en/what-is-the-difference-between-anova-and-regression-and-which-one-to-choose/

If the categorical predictor has only 2 levels such as sex (male, female), then the simple regression analysis is equivalent to an independent t test. If the single categorical variable has more than 2 levels, then the simple linear regression is equivalent to 1-way analysis of variance (ANOVA). If we have 2 categorical predictor variables with 2 or more levels, linear regression is equivalent to 2-way or a higher level of ANOVA. This flexibility of the regression models allows us to perform most analyses using a unified approach. Using linear regression instead of a t test or ANOVA allows us to directly obtain estimates (differences between treatment groups) along with their confidence intervals instead of only P values.
- https://www.ajodo.org/article/S0889-5406(16)00087-1/fulltext#relatedArticles

The main benefit of ANOVA ovethe r regression, in my opinion, is in the output. If you are interested in the statistical significance of the categorical variable (factor) as a block, then ANOVA provides this test for you. With regression, the categorical variable is represented by 2 or more dummy variables, depending on the number of categories, and hence you have 2 or more statistical tests, each comparing the mean for the particular category against the mean of the null category (or the overall mean, depending on dummy coding method). Neither of these may be of interest. Thus, you must perform post-estimation analysis (essentially, ANOVA) to get the overall test of the factor that you are interested in.
- https://stats.stackexchange.com/questions/190984/anova-vs-multiple-linear-regression-why-is-anova-so-commonly-used-in-experiment/191141#191141

When to use the z-test (proportions) vs t-tests (means)?
- https://bloomingtontutors.com/blog/when-to-use-the-z-test-versus-t-test

The usual one and two-sample proportions tests are of this form, and thus we have some justification for treating them as asymptotically normal, but we have no justification for treating them as t-distributed.
In practice, as long as np and n(1−p) are not too small**, the asymptotic normality of the one and two-sample proportions tests comes in very rapidly (that is, often surprisingly small n is enough for both theorems to 'kick in' as it were and the asymptotic behavior to be a good approximation to small sample behavior).
- https://stats.stackexchange.com/questions/90893/why-use-a-z-test-rather-than-a-t-test-with-proportional-data

Is there a minimum sample size required for the t-test to be valid?
Historically, the very first demonstration of the t-test (in "Student"'s 1908 paper) was in an application to sample sizes of size four. Indeed, obtaining improved results for small samples is the test's claim to fame: once the sample size reaches 40 or so, the t-test is not substantially different from the z-tests researchers had been applying throughout the 19th century. 
- https://stats.stackexchange.com/questions/37993/is-there-a-minimum-sample-size-required-for-the-t-test-to-be-valid
- ...but you do have Power concerns when doing t-tests on small samples sizes. 
Eg. the statistical power for a sample size of 15, a one-sample t-test, standard α=.05, and three different effect sizes of .2, .5, .8 (which have sometimes been referred to as small, medium, and large effects respectively) are 11%, 44% and 82% respectively. Only the last one could be said to be "reasonable" power.

Is there a minimum sample size for bootstrapping to be valid?
In short: not really, it can work even for small sample sizes (eg. 20 obs); if they’re not unreasonably small (eg. 4)
- https://stats.stackexchange.com/questions/33300/determining-sample-size-necessary-for-bootstrap-method-proposed-method

Power Analysis For Sample Size Using Python
- https://grabngoinfo.com/power-analysis-for-sample-size-using-python/